## Provision infrastructure with Terraform

Now that everything is set up, we are ready to provision our VM resources with Terraform! We will use Terraform to provision 3 VM instances and associated network resources on the OpenStack cloud.

### Preliminaries

Let’s navigate to the directory with the Terraform configuration for our KVM deployment:

In [23]:
cd /work/MLOps/continous_X_pipeline/tf/kvm/

and make sure we’ll be able to run the terraform executable by adding the directory in which it is located to our PATH:

In [24]:
export PATH=/work/.local/bin:$PATH

We also need to un-set some OpenStack-related environment variables that are set automatically in the Chameleon Jupyter environment, since these will override some Terraform settings that we *don’t* want to override:

In [25]:
unset $(set | grep -o "^OS_[A-Za-z0-9_]*")

### Understanding our Terraform configuration

[data.tf](https://github.com/exploring-curiosity/MLOps/blob/main/continous_X_pipeline/tf/kvm/data.tf) :  data sources gets existing infrastructure details from OpenStack about resources *not* managed by Terraform.

[main.tf](https://github.com/exploring-curiosity/MLOps/blob/main/continous_X_pipeline/tf/kvm/main.tf) :
Here we actually allocate the resources. Except for the block storage, which is previously allocated by the data team member and we have to attach it to node1. We are attaching it to node1 because in our kubernetes cluster, node1 will be the leader node. We are doing this because we want to persist data beyond the lifecycle of our VM instances. This persistent block store will have the storage for MinIO, PostgreSQL, etc.

[variables.tf](https://github.com/exploring-curiosity/MLOps/blob/main/continous_X_pipeline/tf/kvm/variables.tf) : lets us define inputs and reuse the configuration across different environments. The value of variables can be passed in the command line arguments when we run a `terraform` command, or by defining environment variables that start with `TF_VAR`. In this example, there’s a variable `instance_hostname` so that we can re-use this configuration to create a VM with any hostname - the variable is used inside the resource block with `name = "${var.instance_hostname}"`.

### Applying our Terraform configuration

First, we need Terraform to set up our working directory, make sure it has “provider” plugins to interact with our infrastructure provider (it will read in `provider.tf` to check), and set up storage for keeping track of the infrastructure state:

In [27]:
terraform init

Initializing the backend...
Initializing provider plugins...
- Reusing previous version of terraform-provider-openstack/openstack from the dependency lock file
- Using previously-installed terraform-provider-openstack/openstack v1.51.1

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


To follow the project naming conventions and adding the key. Not this is the name of the key followed by the entire team members.

In [28]:
# runs in Chameleon Jupyter environment
export TF_VAR_suffix=project38
export TF_VAR_key=id_rsa_chameleon_project_g38

We should confirm that our planned configuration is valid:

In [31]:
terraform validate

Success! The configuration is valid.



Finally, we will apply those changes. (We need to add an `-auto-approve` argument because ordinarily, Terraform prompts the user to type “yes” to approve the changes it will make.)

In [32]:
terraform apply -auto-approve

data.openstack_networking_secgroup_v2.allow_4000: Reading...
data.openstack_networking_secgroup_v2.allow_3001: Reading...
data.openstack_networking_secgroup_v2.allow_http_80: Reading...
data.openstack_networking_secgroup_v2.allow_8081: Reading...
data.openstack_networking_secgroup_v2.allow_5000: Reading...
data.openstack_networking_secgroup_v2.allow_ssh: Reading...
data.openstack_networking_secgroup_v2.allow_9090: Reading...
data.openstack_networking_network_v2.sharednet2: Reading...
data.openstack_networking_secgroup_v2.allow_9001: Reading...
data.openstack_networking_subnet_v2.sharednet2_subnet: Reading...
data.openstack_networking_secgroup_v2.allow_5000: Read complete after 1s [id=e429f71c-711a-49c2-bfbd-223eccf40f9e]
data.openstack_networking_secgroup_v2.allow_http_80: Read complete after 1s [id=5df2db36-79f9-4b9b-b6c9-1ece762d83fb]
data.openstack_networking_floatingip_v2.reserved_ip: Reading...
data.openstack_networking_secgroup_v2.allow_ssh: Read complete after 1s [id=47860acf-92